<a href="https://colab.research.google.com/github/raywu60kg/Hortonworks-Data-Platform-docker-setup/blob/master/Titanic/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Titanic
Simple example for kaggle titanic

### 1. Load data

In [1]:
# Put our kaggle api token (kaggle.json)
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [2]:
import json
with open('kaggle.json') as f:
  kaggle_api = json.load(f)

In [3]:
import os
os.environ['KAGGLE_USERNAME'] = kaggle_api['username']
os.environ['KAGGLE_KEY'] = kaggle_api['key']

In [4]:
!kaggle competitions download -c titanic

  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 2.59MB/s]
  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 48.6MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 28.4MB/s]


## 2. Check the data




In [5]:
import pandas as pd

In [6]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
submisstion_example = pd.read_csv('gender_submission.csv')

In [7]:
print(len(train_data))
print(train_data.columns)

891
Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


In [8]:
train_data.head()

PassengerId  Survived  Pclass  ...     Fare Cabin  Embarked
0            1         0       3  ...   7.2500   NaN         S
1            2         1       1  ...  71.2833   C85         C
2            3         1       3  ...   7.9250   NaN         S
3            4         1       1  ...  53.1000  C123         S
4            5         0       3  ...   8.0500   NaN         S

[5 rows x 12 columns]

In [9]:
test_data.head()

PassengerId  Pclass  ... Cabin Embarked
0          892       3  ...   NaN        Q
1          893       3  ...   NaN        S
2          894       2  ...   NaN        Q
3          895       3  ...   NaN        S
4          896       3  ...   NaN        S

[5 rows x 11 columns]

## 3. Feature engineering

### 3.1 Remove some features


In [10]:
raw_train_x = train_data.drop(['PassengerId', 'Survived', 'Name', 'Ticket'], axis=1)

In [11]:
raw_test_x = test_data.drop(['PassengerId', 'Name', 'Ticket'], axis=1)

In [12]:
raw_train_x.head()

Pclass     Sex   Age  SibSp  Parch     Fare Cabin Embarked
0       3    male  22.0      1      0   7.2500   NaN        S
1       1  female  38.0      1      0  71.2833   C85        C
2       3  female  26.0      0      0   7.9250   NaN        S
3       1  female  35.0      1      0  53.1000  C123        S
4       3    male  35.0      0      0   8.0500   NaN        S

In [13]:
raw_test_x.head()

Pclass     Sex   Age  SibSp  Parch     Fare Cabin Embarked
0       3    male  34.5      0      0   7.8292   NaN        Q
1       3  female  47.0      1      0   7.0000   NaN        S
2       2    male  62.0      0      0   9.6875   NaN        Q
3       3    male  27.0      0      0   8.6625   NaN        S
4       3  female  22.0      1      1  12.2875   NaN        S

In [14]:
train_y = train_data['Survived']

In [15]:
train_y.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

### 3.2 Parse individually

 ##### 3.2.1 Pclass





In [16]:
import numpy as np

In [17]:
print('Train data has ', sum(np.isnan(raw_train_x['Pclass'])), 'nan in Pclass.')
print('Test data has ', sum(np.isnan(raw_test_x['Pclass'])), 'nan in Pclass.')

Train data has  0 nan in Pclass.
Test data has  0 nan in Pclass.


In [18]:
train_pclass = pd.get_dummies(raw_train_x['Pclass'])
test_pclass = pd.get_dummies(raw_test_x['Pclass'])

In [19]:
train_pclass.columns = ['pclass_1', 'pclass_2','pclass_3']
test_pclass.columns = ['pclass_1', 'pclass_2','pclass_3']

In [20]:
train_pclass

pclass_1  pclass_2  pclass_3
0           0         0         1
1           1         0         0
2           0         0         1
3           1         0         0
4           0         0         1
..        ...       ...       ...
886         0         1         0
887         1         0         0
888         0         0         1
889         1         0         0
890         0         0         1

[891 rows x 3 columns]

In [21]:
test_pclass

pclass_1  pclass_2  pclass_3
0           0         0         1
1           0         0         1
2           0         1         0
3           0         0         1
4           0         0         1
..        ...       ...       ...
413         0         0         1
414         1         0         0
415         0         0         1
416         0         0         1
417         0         0         1

[418 rows x 3 columns]

##### 3.2.2 Sex



In [22]:
train_sex = pd.get_dummies(raw_train_x['Sex'])
test_sex = pd.get_dummies(raw_test_x['Sex'])

In [23]:
train_sex.columns = ['sex_female', 'sex_male']
test_sex.columns = ['sex_female', 'sex_male']

In [24]:
train_sex

sex_female  sex_male
0             0         1
1             1         0
2             1         0
3             1         0
4             0         1
..          ...       ...
886           0         1
887           1         0
888           1         0
889           0         1
890           0         1

[891 rows x 2 columns]

In [25]:
test_sex

sex_female  sex_male
0             0         1
1             1         0
2             0         1
3             0         1
4             1         0
..          ...       ...
413           0         1
414           1         0
415           0         1
416           0         1
417           0         1

[418 rows x 2 columns]

##### 3.2.3 Age

In [26]:
print('Train data has ', sum(np.isnan(raw_train_x['Age'])), 'nan in Age.')
print('Test data has ', sum(np.isnan(raw_test_x['Age'])), 'nan in Age.')

Train data has  177 nan in Age.
Test data has  86 nan in Age.


In [27]:
train_age = raw_train_x['Age'].fillna(np.mean(raw_train_x['Age']))
test_age = raw_test_x['Age'].fillna(np.mean(raw_train_x['Age']))

In [28]:
train_age

0      22.000000
1      38.000000
2      26.000000
3      35.000000
4      35.000000
         ...    
886    27.000000
887    19.000000
888    29.699118
889    26.000000
890    32.000000
Name: Age, Length: 891, dtype: float64

In [29]:
test_age

0      34.500000
1      47.000000
2      62.000000
3      27.000000
4      22.000000
         ...    
413    29.699118
414    39.000000
415    38.500000
416    29.699118
417    29.699118
Name: Age, Length: 418, dtype: float64

##### 3.2.4 SibSp

In [30]:
print('Train data has ', sum(np.isnan(raw_train_x['SibSp'])), 'nan in SibSp.')
print('Test data has ', sum(np.isnan(raw_test_x['SibSp'])), 'nan in SibSp.')

Train data has  0 nan in SibSp.
Test data has  0 nan in SibSp.


In [31]:
train_sibsp = raw_train_x['SibSp']
test_sibsp = raw_test_x['SibSp']

In [32]:
train_sibsp

0      1
1      1
2      0
3      1
4      0
      ..
886    0
887    0
888    1
889    0
890    0
Name: SibSp, Length: 891, dtype: int64

In [33]:
test_sibsp

0      0
1      1
2      0
3      0
4      1
      ..
413    0
414    0
415    0
416    0
417    1
Name: SibSp, Length: 418, dtype: int64

##### 3.2.5 Parch

In [34]:
print('Train data has ', sum(np.isnan(raw_train_x['Parch'])), 'nan in Parch.')
print('Test data has ', sum(np.isnan(raw_test_x['Parch'])), 'nan in Parch.')

Train data has  0 nan in Parch.
Test data has  0 nan in Parch.


In [35]:
train_parch = raw_train_x['Parch']
test_parch = raw_test_x['Parch']

In [36]:
train_parch

0      0
1      0
2      0
3      0
4      0
      ..
886    0
887    0
888    2
889    0
890    0
Name: Parch, Length: 891, dtype: int64

In [37]:
test_parch

0      0
1      0
2      0
3      0
4      1
      ..
413    0
414    0
415    0
416    0
417    1
Name: Parch, Length: 418, dtype: int64

##### 3.2.6 Cabin	

In [38]:
raw_train_x['Cabin']

0       NaN
1       C85
2       NaN
3      C123
4       NaN
       ... 
886     NaN
887     B42
888     NaN
889    C148
890     NaN
Name: Cabin, Length: 891, dtype: object

In [39]:
tmp_cabin = []
for index in raw_train_x['Cabin']:
  if str(index) == 'nan':
    tmp_cabin.append(np.nan)
  else:
    tmp_cabin.append(str(index)[0])

In [40]:
train_cabin = pd.get_dummies(tmp_cabin, dummy_na=True)
train_cabin

A  B  C  D  E  F  G  T  NaN
0    0  0  0  0  0  0  0  0    1
1    0  0  1  0  0  0  0  0    0
2    0  0  0  0  0  0  0  0    1
3    0  0  1  0  0  0  0  0    0
4    0  0  0  0  0  0  0  0    1
..  .. .. .. .. .. .. .. ..  ...
886  0  0  0  0  0  0  0  0    1
887  0  1  0  0  0  0  0  0    0
888  0  0  0  0  0  0  0  0    1
889  0  0  1  0  0  0  0  0    0
890  0  0  0  0  0  0  0  0    1

[891 rows x 9 columns]

In [41]:
tmp_cabin = []
for index in raw_test_x['Cabin']:
  if str(index) == 'nan':
    tmp_cabin.append(np.nan)
  else:
    tmp_cabin.append(str(index)[0])

In [42]:
test_cabin = pd.get_dummies(tmp_cabin, dummy_na=True)
test_cabin

A  B  C  D  E  F  G  NaN
0    0  0  0  0  0  0  0    1
1    0  0  0  0  0  0  0    1
2    0  0  0  0  0  0  0    1
3    0  0  0  0  0  0  0    1
4    0  0  0  0  0  0  0    1
..  .. .. .. .. .. .. ..  ...
413  0  0  0  0  0  0  0    1
414  0  0  1  0  0  0  0    0
415  0  0  0  0  0  0  0    1
416  0  0  0  0  0  0  0    1
417  0  0  0  0  0  0  0    1

[418 rows x 8 columns]

In [43]:
# add 'T' in test
test_cabin['T'] = np.repeat(0, len(test_cabin))
test_cabin.columns

Index(['A', 'B', 'C', 'D', 'E', 'F', 'G', nan, 'T'], dtype='object')

In [44]:
columnsTitles=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'T',np.nan]
test_cabin=test_cabin.reindex(columns=columnsTitles)
test_cabin

A  B  C  D  E  F  G  T  NaN
0    0  0  0  0  0  0  0  0    1
1    0  0  0  0  0  0  0  0    1
2    0  0  0  0  0  0  0  0    1
3    0  0  0  0  0  0  0  0    1
4    0  0  0  0  0  0  0  0    1
..  .. .. .. .. .. .. .. ..  ...
413  0  0  0  0  0  0  0  0    1
414  0  0  1  0  0  0  0  0    0
415  0  0  0  0  0  0  0  0    1
416  0  0  0  0  0  0  0  0    1
417  0  0  0  0  0  0  0  0    1

[418 rows x 9 columns]

In [45]:
train_cabin.columns = [
  'cabin_A', 'cabin_B', 'cabin_C', 'cabin_D', 'cabin_E', 'cabin_F', 'cabin_G', 'cabin_T','cabin_nan']
test_cabin.columns = [
  'cabin_A', 'cabin_B', 'cabin_C', 'cabin_D', 'cabin_E', 'cabin_F', 'cabin_G', 'cabin_T','cabin_nan']

In [46]:
train_cabin

cabin_A  cabin_B  cabin_C  cabin_D  ...  cabin_F  cabin_G  cabin_T  cabin_nan
0          0        0        0        0  ...        0        0        0          1
1          0        0        1        0  ...        0        0        0          0
2          0        0        0        0  ...        0        0        0          1
3          0        0        1        0  ...        0        0        0          0
4          0        0        0        0  ...        0        0        0          1
..       ...      ...      ...      ...  ...      ...      ...      ...        ...
886        0        0        0        0  ...        0        0        0          1
887        0        1        0        0  ...        0        0        0          0
888        0        0        0        0  ...        0        0        0          1
889        0        0        1        0  ...        0        0        0          0
890        0        0        0        0  ...        0        0        0          1

[891 rows x 9 columns]

In [47]:
test_cabin

cabin_A  cabin_B  cabin_C  cabin_D  ...  cabin_F  cabin_G  cabin_T  cabin_nan
0          0        0        0        0  ...        0        0        0          1
1          0        0        0        0  ...        0        0        0          1
2          0        0        0        0  ...        0        0        0          1
3          0        0        0        0  ...        0        0        0          1
4          0        0        0        0  ...        0        0        0          1
..       ...      ...      ...      ...  ...      ...      ...      ...        ...
413        0        0        0        0  ...        0        0        0          1
414        0        0        1        0  ...        0        0        0          0
415        0        0        0        0  ...        0        0        0          1
416        0        0        0        0  ...        0        0        0          1
417        0        0        0        0  ...        0        0        0          1

[418 rows x 9 columns]

##### 3.2.7 Embarked


In [48]:
train_embarked = pd.get_dummies(raw_train_x['Embarked'], dummy_na=True)
test_embarked = pd.get_dummies(raw_test_x['Embarked'], dummy_na=True)

In [49]:
train_embarked.columns = ['embarked_C', 'embarked_Q', 'embarked_S', 'embarked_nan']
test_embarked.columns = ['embarked_C', 'embarked_Q', 'embarked_S', 'embarked_nan']

In [50]:
train_embarked

embarked_C  embarked_Q  embarked_S  embarked_nan
0             0           0           1             0
1             1           0           0             0
2             0           0           1             0
3             0           0           1             0
4             0           0           1             0
..          ...         ...         ...           ...
886           0           0           1             0
887           0           0           1             0
888           0           0           1             0
889           1           0           0             0
890           0           1           0             0

[891 rows x 4 columns]

In [51]:
test_embarked

embarked_C  embarked_Q  embarked_S  embarked_nan
0             0           1           0             0
1             0           0           1             0
2             0           1           0             0
3             0           0           1             0
4             0           0           1             0
..          ...         ...         ...           ...
413           0           0           1             0
414           1           0           0             0
415           0           0           1             0
416           0           0           1             0
417           1           0           0             0

[418 rows x 4 columns]

## Train Test Split


In [52]:
train_x = pd.concat([train_pclass, train_sex, train_age, train_sibsp, train_parch, train_cabin, train_embarked], axis=1)
test_x = pd.concat([test_pclass, test_sex, test_age, test_sibsp, test_parch, test_cabin, test_embarked], axis=1)

In [53]:
train_x

pclass_1  pclass_2  pclass_3  ...  embarked_Q  embarked_S  embarked_nan
0           0         0         1  ...           0           1             0
1           1         0         0  ...           0           0             0
2           0         0         1  ...           0           1             0
3           1         0         0  ...           0           1             0
4           0         0         1  ...           0           1             0
..        ...       ...       ...  ...         ...         ...           ...
886         0         1         0  ...           0           1             0
887         1         0         0  ...           0           1             0
888         0         0         1  ...           0           1             0
889         1         0         0  ...           0           0             0
890         0         0         1  ...           1           0             0

[891 rows x 21 columns]

In [54]:
test_x

pclass_1  pclass_2  pclass_3  ...  embarked_Q  embarked_S  embarked_nan
0           0         0         1  ...           1           0             0
1           0         0         1  ...           0           1             0
2           0         1         0  ...           1           0             0
3           0         0         1  ...           0           1             0
4           0         0         1  ...           0           1             0
..        ...       ...       ...  ...         ...         ...           ...
413         0         0         1  ...           0           1             0
414         1         0         0  ...           0           0             0
415         0         0         1  ...           0           1             0
416         0         0         1  ...           0           1             0
417         0         0         1  ...           0           0             0

[418 rows x 21 columns]

In [55]:
from sklearn.model_selection import train_test_split
train_x, eval_x, train_y, eval_y = train_test_split(train_x, train_y, test_size=0.2, random_state=0)

## 4. Modeling

### 4.1 xgb model

In [56]:
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

xgb_model = xgb.XGBClassifier(objective="binary:logistic",n_jobs=-1, random_state=0)
xgb_model.fit(train_x, train_y)

y_pred = xgb_model.predict(eval_x)

print(accuracy_score(eval_y, y_pred))

0.8379888268156425


In [57]:
xgb_model.get_params

<bound method XGBModel.get_params of XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=-1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)>

### 4.2 Search hyper params

In [58]:
parameters = {
    'learning_rate':[0.1, 0.05, 0.01, 0.005],
     'gamma':[0.1, 0.05, 0.01, 0.005],
     'max_depth':[2, 3, 4, 5],
     'n_estimators':[100, 150, 200, 250, 300, 400, 500]
}

In [59]:
xgb_model = xgb.XGBClassifier(objective="binary:logistic")
clf = GridSearchCV(xgb_model, parameters, scoring='accuracy', n_jobs=-1, cv=3)
clf.fit(train_x, train_y)

GridSearchCV(cv=3, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=-1,
             param_grid={'gamma': [0.1, 0.05, 0.01, 0.005],
                         'learning_rate': [0.1, 0.05, 0.01, 0.005],
                         'max_depth': [2, 3, 4, 5],

In [60]:
print(clf.best_params_)
print(clf.best_score_)

{'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 100}
0.8300771785507451


## 5. Predict

### 5.1 Evaluate

In [61]:
xgb_model = xgb.XGBClassifier(
    objective="binary:logistic",
    n_jobs=-1,
    gamma=clf.best_params_['gamma'],
    learning_rate=clf.best_params_['learning_rate'],
    max_depth=clf.best_params_['max_depth'],
    n_estimators=clf.best_params_['n_estimators']
)
xgb_model.fit(train_x, train_y)

pred_y = xgb_model.predict(eval_x)

print(accuracy_score(eval_y, pred_y))

0.8268156424581006


### 5.2 Predict test_x


In [62]:
pred_y = xgb_model.predict(test_x)
print(pred_y)

[0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 0 0
 1 0 0 1 0 1 1 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0
 0 0 1 0 0 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 1 0 1 1 0 1
 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 1 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0 1 1 0 1 0 0 0 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 1 1 1 1 0 0 1 0 0 0]


## Submit


In [63]:
submisstion_example

PassengerId  Survived
0            892         0
1            893         1
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]

In [64]:
submission = pd.DataFrame({'PassengerId':test_data['PassengerId'], 'Survived':pred_y})
submission

PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         0
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]

In [65]:
submission.to_csv('/content/submission.csv', index=False)

In [66]:
!ls

gender_submission.csv  kaggle.json  submission.csv  test.csv  train.csv


In [67]:
!kaggle competitions submit  -f submission.csv -m 'my_submisstion' titanic

100% 2.77k/2.77k [00:03<00:00, 733B/s]
Successfully submitted to Titanic: Machine Learning from Disaster